In [1]:
import pandas as pd
import sqlite3
import os
from glob import glob
import re

In [5]:
conn = sqlite3.connect("interwencje_psp.db")
cursor = conn.cursor()

cursor.execute("""
DROP TABLE IF EXISTS "3_2"
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS "3_2" (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    rodzaj_obiektu TEXT,
    obiekt_szczegol TEXT,
    wojewodztwo TEXT,
    powiat TEXT,
    gmina TEXT,
    liczba INTEGER,
    rok INTEGER
)
""")

In [7]:
conn = sqlite3.connect("interwencje_psp.db")

def process_old_format(df_raw, year):
    """Przetwarzanie starego formatu (2016-2022)"""
    results = []
    current_voivodeship = None
    current_powiat = None
    current_gmina = None

    categories = {}
    current_main_category = None

    for col in range(2, df_raw.shape[1]):
        main_cat = df_raw.iloc[3, col]
        sub_cat = df_raw.iloc[4, col]
        
        if pd.notna(main_cat) and "RAZEM" not in str(main_cat):
            current_main_category = main_cat
        if pd.notna(sub_cat):
            categories[col] = (current_main_category, sub_cat)

    for i in range(5, df_raw.shape[0]):
        row = df_raw.iloc[i]
        
        if isinstance(row[1], str):
            if row[1].isupper():
                current_voivodeship = row[1]
                current_powiat = None
                current_gmina = None
            elif "gmina" in row[1].lower():
                current_gmina = row[1]
            elif row[1][0].isupper() and current_voivodeship:
                current_powiat = row[1].lower()
        
        if not current_voivodeship:
            continue
            
        for col in categories:
            try:
                value = row[col]
                if pd.notna(value) and str(value).isdigit() and int(value) > 0:
                    main_cat, sub_cat = categories[col]
                    results.append([
                        main_cat,         
                        current_voivodeship,
                        current_powiat,  
                        current_gmina,   
                        sub_cat,         
                        int(value),       
                        year             
                    ])
            except (ValueError, AttributeError):
                continue

    return results
def process_new_format(df_raw, year):
    """Przetwarzanie nowego formatu (2023-2024)"""
    results = []
    
    main_categories = [] 
    column_mapping = {} 
    
    current_main_category = None
    
    for col in df_raw.columns:
        if isinstance(col, str):
            is_main_category = False
            for i in range(1, 10): 
                if col.startswith(f'{i}. '):
                    current_main_category = col
                    main_categories.append(col)
                    is_main_category = True
                    break
            
            if col.startswith('RAZEM ') and not is_main_category:
                current_main_category = col.replace('RAZEM ', '').strip()
                main_categories.append(current_main_category)
                is_main_category = True
            
            if not is_main_category and current_main_category:
                if col not in ['TERYT', 'Województwo', 'Powiat', 'Gmina'] and \
                   not col.startswith('OGÓŁEM') and not col.startswith('RAZEM'):
                    column_mapping[col] = (current_main_category, col)
    
    # print(f"Znalezione główne kategorie: {main_categories}")
    # print(f"Znalezione podkategorie: {len(column_mapping)}")
    
    for _, row in df_raw.iterrows():
        wojewodztwo = row["Województwo"]
        powiat = row["Powiat"]
        gmina = row["Gmina"]
        
        for col_name, (rodzaj_obiektu, obiekt_szczegol) in column_mapping.items():
            if col_name in row:
                value = row[col_name]
                if pd.notna(value) and value > 0:
                    results.append([
                        rodzaj_obiektu,   
                        wojewodztwo,       
                        powiat,        
                        gmina,            
                        obiekt_szczegol,  
                        int(value),       
                        year            
                    ])
    
    return results

for year in range(2016, 2025):
    folder = os.path.join("../raw_data", str(year))
    
    if year == 2023:
        patterns = [os.path.join(folder, "3_2023.xlsx")]
    elif year == 2024:
        patterns = [os.path.join(folder, "3.xlsx")]
    else:
        patterns = [
            os.path.join(folder, f"3_2_{year}.xlsx"),
            os.path.join(folder, "3_2_*.xlsx"), 
            os.path.join(folder, "3_2.*.xlsx")
        ]
    
    matches = []
    for pattern in patterns:
        matches.extend(glob(pattern))
        if matches:  
            break
    
    if not matches:
        print(f"Brak odpowiedniego pliku w folderze {year}, pomijam.")
        continue

    file_path = matches[0]
    print(f"Przetwarzanie pliku: {file_path}")

    df_raw = pd.read_excel(file_path, header=None if year < 2023 else 0)
    
    if year >= 2023:
        if "Województwo" in df_raw.columns:
            print(f"Rok {year}: wykryto nowy format danych")
            records = process_new_format(df_raw, year)
        else:
            print(f"Rok {year}: nierozpoznany format, próbuję stary format")
            df_raw = pd.read_excel(file_path, header=None)
            records = process_old_format(df_raw, year)
    else:
        records = process_old_format(df_raw, year)

    if records:
        df_final = pd.DataFrame(records, columns=["rodzaj_obiektu", "wojewodztwo", "powiat", "gmina", "obiekt_szczegol", "liczba", "rok"])
        df_final.to_sql("3_2", conn, if_exists="append", index=False)
        print(f"Dodano {len(df_final)} rekordów z roku {year} do tabeli '3_2'.\n")
    else:
        print(f"Rok {year}: brak rekordów do zapisania — możliwe inne formatowanie pliku.\n")

conn.commit()
conn.close()

print("Wszystkie dostępne dane zostały zapisane do bazy obiekty_psp.db.")

Przetwarzanie pliku: ../raw_data\2016\3_2_2016.xlsx
Dodano 59204 rekordów z roku 2016 do tabeli '3_2'.

Przetwarzanie pliku: ../raw_data\2017\3_2_2017.xlsx
Dodano 57334 rekordów z roku 2017 do tabeli '3_2'.

Przetwarzanie pliku: ../raw_data\2018\3_2. Pożary wg kodu obiektu w rozbiciu na województwo, powiat, gminę.xlsx
Dodano 63019 rekordów z roku 2018 do tabeli '3_2'.

Przetwarzanie pliku: ../raw_data\2019\3_2_Pożary wg kodu obiektu w rozbiciu na województwo, powiat, gminę.xlsx
Dodano 63279 rekordów z roku 2019 do tabeli '3_2'.

Przetwarzanie pliku: ../raw_data\2020\3_2_Po¥ary wg kodu obiektu w rozbiciu na województwo, powiat, gmin®.xlsx
Dodano 59890 rekordów z roku 2020 do tabeli '3_2'.

Przetwarzanie pliku: ../raw_data\2021\3_2_2021.xlsx
Dodano 57616 rekordów z roku 2021 do tabeli '3_2'.

Przetwarzanie pliku: ../raw_data\2022\3_2_2022.xlsx
Dodano 61325 rekordów z roku 2022 do tabeli '3_2'.

Przetwarzanie pliku: ../raw_data\2023\3_2023.xlsx
Rok 2023: wykryto nowy format danych
Dodano 

In [8]:
conn = sqlite3.connect("interwencje_psp.db")
# df = pd.read_sql_query('SELECT DISTINCT rok FROM "1_2" ORDER BY rok ASC', conn)
df = pd.read_sql_query('''SELECT rodzaj_obiektu, obiekt_szczegol FROM "3_2" WHERE rok = 2023 ''', conn)
print(df.head(10))
conn.close()

                       rodzaj_obiektu  \
0  1. Obiekty użyteczności publicznej   
1  1. Obiekty użyteczności publicznej   
2  1. Obiekty użyteczności publicznej   
3  1. Obiekty użyteczności publicznej   
4  1. Obiekty użyteczności publicznej   
5  1. Obiekty użyteczności publicznej   
6  1. Obiekty użyteczności publicznej   
7  1. Obiekty użyteczności publicznej   
8               2. Obiekty mieszkalne   
9               2. Obiekty mieszkalne   

                                     obiekt_szczegol  
0                101. Administracyjno-biurowe, banki  
1  102. Oświaty i nauki, w szczególności budynki ...  
2  103. Służby zdrowia, w szczególności szpitale,...  
3  104. Handlowo-usługowe, w szczególności sklepy...  
4  105. Obsługi pasażerów w komunikacji, w szczeg...  
5             106. Widowiskowo-rozrywkowe i sportowe  
6                   107. Kultu religijnego, sakralne  
7          111. Inne obiekty użyteczności publicznej  
8                           201. Hotele, noclegownie 